In [1]:
import sqlite3
import pandas as pd
from pandas import DataFrame
from functools import reduce
from numpy import array
import numpy as np

In [2]:
#this connects to sqlite db
conn = sqlite3.connect("/Users/user/Desktop/Research Projects/Coding_and_Analysis/imdb.db")

In [3]:
#test connection and reformats data as a dataframe with indexed data
df = pd.read_sql_query("SELECT * from encyclopedia \
LEFT OUTER JOIN linktbl on linktbl.uid = encyclopedia.uid", conn)
df = DataFrame(df)
df.columns = df.keys()

In [4]:
df.nunique()

uid                    1732
program_title          1672
program_type             10
program_genre            30
network                  57
first_air_month          13
first_air_day            31
first_air_year           13
last_air_month            3
last_air_day              3
last_air_year            37
program_description    1728
uid                    1278
id                     1183
dtype: int64

In [5]:
#this query pulls info_type_id 98,3,17,10. These are plot(98), genre(3), keywords(10),trivia(17). 
# NB: I got no values for 10. Also I ultimately dropped the notes column as it looks useless.
#This cell also reformats query into a dataframe and renames second id field as lid
df1 = pd.read_sql_query("SELECT * from movie_info2 \
INNER JOIN linktbl on linktbl.id = movie_info2.movie_id \
WHERE movie_info2.info_type_id = 98 \
OR movie_info2.info_type_id = 3 \
OR movie_info2.info_type_id = 17 \
OR movie_info2.info_type_id = 10", conn)
df1 = DataFrame(df1)
df1.columns = df1.keys()
df1.columns = ['id','movie_id','info_type_id','info','note','uid','lid']

In [6]:
df1.nunique()

id              4239
movie_id        1123
info_type_id       3
info            2487
note             106
uid             1227
lid             1123
dtype: int64

In [7]:
df1['note'].count()

439

In [8]:
#this creates a pivot table from info_type_id, creating 3 columns, turning everything else into rows
df_info = df1.pivot_table(values='info', index=['id', 'movie_id', 'uid', 'lid'], columns= 'info_type_id', \
                aggfunc='last')

In [9]:
df_info.count()

info_type_id
3     2112
17    2862
98     439
dtype: int64

In [10]:
#reseting the index
df_info.reset_index(inplace=True)

In [11]:
df_info.count()

info_type_id
id          5413
movie_id    5413
uid         5413
lid         5413
3           2112
17          2862
98           439
dtype: int64

In [12]:
#this cell drops extraneous columns (id, lid, uid, etc), while grouping 3,98 and 17 by movie_id. 
# This essence we are combining all items associated with like movie_ids into a single cell. 
gb = df_info.groupby(('movie_id'))
result = gb[3].unique()
result1 = gb[98].unique()
result2 = gb[17].unique()

In [13]:
#in the next three cells, we create a dataframe from the results and print them. movie_id is the index.
result = pd.DataFrame({'movie_id':result.index, '3':result.values})
result.sort_index().head()

,3,movie_id
0,"[None, Drama]",16535
1,"[None, Western]",18439
2,"[None, Action, Crime, Drama]",35319
3,"[None, Comedy, Music]",41569
4,[Comedy],42033


In [14]:
result1 = pd.DataFrame({'movie_id':result1.index, '98':result1.values})
result1.sort_index().head()

,98,movie_id
0,"[None, Private investigator Dennis Chase opera...",16535
1,[None],18439
2,"[None, Stu Bailey and Jeff Spencer were the wi...",35319
3,[None],41569
4,[None],42033


In [15]:
result2 = pd.DataFrame({'movie_id':result2.index, '17':result2.values})
result2.sort_index().head()

,17,movie_id
0,"[""21 Beacon Street"" was the address of Dennis ...",16535
1,"['Lane Bradford' (qv), who usually played heav...",18439
2,"[The restaurant, Dino's Lodge, was then owned ...",35319
3,"[This mish-mash of variety, music and prize gi...",41569
4,[None],42033


In [16]:
#this cell combines the results and merges them by movie_id
dfs = [result, result1, result2]
df_final = reduce(lambda left,right: pd.merge(left,right,on='movie_id'), dfs)

In [17]:
df_final.head()

,3,movie_id,98,17
0,"[None, Drama]",16535,"[None, Private investigator Dennis Chase opera...","[""21 Beacon Street"" was the address of Dennis ..."
1,"[None, Western]",18439,[None],"['Lane Bradford' (qv), who usually played heav..."
2,"[None, Action, Crime, Drama]",35319,"[None, Stu Bailey and Jeff Spencer were the wi...","[The restaurant, Dino's Lodge, was then owned ..."
3,"[None, Comedy, Music]",41569,[None],"[This mish-mash of variety, music and prize gi..."
4,[Comedy],42033,[None],[None]


In [18]:
#database pull using left outer joins
df2 = pd.read_sql_query("SELECT * from title2 \
LEFT OUTER JOIN movie_companies2 ON title2.id=movie_companies2.movie_id \
LEFT OUTER JOIN company_name2 ON movie_companies2.company_id = company_name2.id \
LEFT OUTER JOIN linktbl on linktbl.id = title2.id \
LEFT OUTER JOIN encyclopedia on encyclopedia.uid = linktbl.uid \
WHERE company_name2.country_code like '[us]' AND title2.kind_id <>7", conn)

In [19]:
df2.nunique()

id                     2397
title                  2326
imdb_index                3
kind_id                   2
production_year          20
imdb_id                   0
phonetic_code          1775
episode_of_id             0
season_nr                 0
episode_nr                0
series_years            238
md5sum                 2397
id                     5744
movie_id               2397
company_id              863
company_type_id           2
note                   1067
id                      863
name                    863
country_code              1
imdb_id                   0
name_pcode_nf           708
name_pcode_sf           702
md5sum                  863
uid                    1264
id                     1165
uid                    1264
program_title          1213
program_type              7
program_genre            27
network                  49
first_air_month          12
first_air_day            23
first_air_year           13
last_air_month            2
last_air_day        

In [20]:
#this column turns df2 into a dataframe, indexes it, drops unnecessary columns and renames it as df3
df2 = DataFrame(df2)
df2.columns = df2.keys()
df3 = df2.drop(columns=['imdb_id', 'imdb_index', 'name', 'phonetic_code', 'episode_of_id', 'season_nr', 'episode_nr', 'md5sum', 'id','company_id', \
         'company_type_id', 'note', 'id', 'country_code', 'imdb_id', 'name_pcode_nf', 'name_pcode_sf', 'md5sum' \
         'id', 'uid', 'first_air_month', 'first_air_day', 'last_air_month', 'last_air_day'])

In [21]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6699 entries, 0 to 6698
Data columns (total 12 columns):
title                  6699 non-null object
kind_id                6699 non-null int64
production_year        6699 non-null int64
series_years           5693 non-null object
movie_id               6699 non-null int64
program_title          4447 non-null object
program_type           4447 non-null object
program_genre          4447 non-null object
network                4447 non-null object
first_air_year         4447 non-null float64
last_air_year          4447 non-null object
program_description    4447 non-null object
dtypes: float64(1), int64(3), object(8)
memory usage: 628.1+ KB


In [22]:
df3.nunique()

title                  2326
kind_id                   2
production_year          20
series_years            238
movie_id               2397
program_title          1213
program_type              7
program_genre            27
network                  49
first_air_year           13
last_air_year            37
program_description    1263
dtype: int64

In [23]:
#this cell performs a left merge between df3 and df_final. It merges on movie_id
frames = pd.merge(df3, df_final, how='left', on=['movie_id', 'movie_id'])

In [24]:
frames

,title,kind_id,production_year,series_years,movie_id,program_title,program_type,program_genre,network,first_air_year,last_air_year,program_description,3,98,17
0,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN
1,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN
2,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN
3,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN
4,1947 World Series,2,1947,1947-????,10916,None,None,None,None,NaN,None,None,NaN,NaN,NaN
5,1947 World Series,2,1947,1947-????,10916,None,None,None,None,NaN,None,None,NaN,NaN,NaN
6,1948 World Series,2,1948,1948-????,10924,None,None,None,None,NaN,None,None,NaN,NaN,NaN
7,1948 World Series,2,1948,1948-????,10924,None,None,None,None,NaN,None,None,NaN,NaN,NaN
8,1948 World Series,2,1948,1948-????,10924,None,None,None,None,NaN,None,None,NaN,NaN,NaN
9,1949 World Series,2,1949,1949-????,10931,None,None,None,None,NaN,None,None,NaN,NaN,NaN


In [25]:
#this cell checks the columns for stray white space
frames.columns

Index(['title', 'kind_id', 'production_year', 'series_years', 'movie_id',
       'program_title', 'program_type', 'program_genre', 'network',
       'first_air_year', 'last_air_year', 'program_description', '3', '98',
       '17'],
      dtype='object')

In [26]:
#these three cells rename columns
frames = frames.rename(columns={'3': 'genre(3)'})

In [27]:
frames = frames.rename(columns={'98': 'plot(98)'})

In [28]:
frames = frames.rename(columns={'17': 'trivia(17)'})

In [29]:
frames.head()

,title,kind_id,production_year,series_years,movie_id,program_title,program_type,program_genre,network,first_air_year,last_air_year,program_description,genre(3),plot(98),trivia(17)
0,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN
1,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN
2,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN
3,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN
4,1947 World Series,2,1947,1947-????,10916,None,None,None,None,NaN,None,None,NaN,NaN,NaN


In [30]:
#This generates the real title list by checking if the program_title string has a length >= 1, if so it populates the column
frames['realtitle'] = np.where(frames['program_title'].str.len()>=1, frames['program_title'], frames['title'])

In [31]:
frames.head()

,title,kind_id,production_year,series_years,movie_id,program_title,program_type,program_genre,network,first_air_year,last_air_year,program_description,genre(3),plot(98),trivia(17),realtitle
0,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN,13 Demon Street
1,13 Demon Street,2,1959,1959-1960,8305,None,None,None,None,NaN,None,None,NaN,NaN,NaN,13 Demon Street
2,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN,13th Hour
3,13th Hour,2,1958,1958-1959,9296,None,None,None,None,NaN,None,None,NaN,NaN,NaN,13th Hour
4,1947 World Series,2,1947,1947-????,10916,None,None,None,None,NaN,None,None,NaN,NaN,NaN,1947 World Series


In [34]:
#reorder columns by typing in the column names. Note the double brackets.
frames = frames[['realtitle','program_type','program_genre','network','program_description','first_air_year','last_air_year','genre(3)','plot(98)','trivia(17)','program_title','movie_id','kind_id','title']]

In [36]:
frames.head(10)

,realtitle,program_type,program_genre,network,program_description,first_air_year,last_air_year,genre(3),plot(98),trivia(17),program_title,movie_id,kind_id,title
0,13 Demon Street,None,None,None,None,NaN,None,NaN,NaN,NaN,None,8305,2,13 Demon Street
1,13 Demon Street,None,None,None,None,NaN,None,NaN,NaN,NaN,None,8305,2,13 Demon Street
2,13th Hour,None,None,None,None,NaN,None,NaN,NaN,NaN,None,9296,2,13th Hour
3,13th Hour,None,None,None,None,NaN,None,NaN,NaN,NaN,None,9296,2,13th Hour
4,1947 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10916,2,1947 World Series
5,1947 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10916,2,1947 World Series
6,1948 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10924,2,1948 World Series
7,1948 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10924,2,1948 World Series
8,1948 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10924,2,1948 World Series
9,1949 World Series,None,None,None,None,NaN,None,NaN,NaN,NaN,None,10931,2,1949 World Series


In [37]:
frames.columns

Index(['realtitle', 'program_type', 'program_genre', 'network',
       'program_description', 'first_air_year', 'last_air_year', 'genre(3)',
       'plot(98)', 'trivia(17)', 'program_title', 'movie_id', 'kind_id',
       'title'],
      dtype='object')

In [357]:
frames = frames.rename(columns=lambda x: x.strip())

In [38]:
frames.to_csv('frames.csv', encoding='utf-8')